<a href="https://colab.research.google.com/github/AJSTYLE-lab/Fundamental-of-BIg-Data-Analysis/blob/main/SPARK%20LABS/LAB%231/L21_7692.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
import random 
from pyspark.sql.functions import length,udf,desc,col
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql import SparkSession

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#QUESTION_ONE:

In [ ]:
arr=[1,2,3,4,5,6,7,8,9,10]
list(map(lambda x:x**2,arr))

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

#QUESTION_TWO:

In [ ]:
LIST=[]
for i in range(0,20):
  LIST.append(random.randint(3,9))
print("RANDOM LIST GENERATED=",LIST)
list(filter(lambda x:(x%5==0),LIST))

RANDOM LIST GENERATED= [4, 5, 9, 6, 9, 6, 4, 7, 6, 9, 9, 3, 3, 3, 5, 3, 3, 6, 7, 8]


[5, 5]

#QUESTION_THREE:

In [ ]:
spark=SparkSession.builder\
.master("local[1]")\
.appName("MY_FIRST_PYSPARK_CODE")\
.getOrCreate()
spark

#LOAD THE FILE:

In [ ]:
FILE=spark.read.csv("DATA_TWO.txt")

#COMMENTS OF EACH USER:

In [ ]:
FILE.show()

+--------+--------------------+
|     _c0|                 _c1|
+--------+--------------------+
|Aliya153| Your website is ...|
|   Sara2| You need to work...|
|   Ali45|            Good !!!|
|   Ali45| I will definitel...|
+--------+--------------------+



#GIVING NAMES TO COLUMNS:

In [ ]:
schema = "USER_NAME STRING, COMMENTS STRING"
FILE = spark.read.format("csv") \
             .schema(schema) \
             .load("DATA_TWO.txt")
FILE.show()

+---------+--------------------+
|USER_NAME|            COMMENTS|
+---------+--------------------+
| Aliya153| Your website is ...|
|    Sara2| You need to work...|
|    Ali45|            Good !!!|
|    Ali45| I will definitel...|
+---------+--------------------+



In [ ]:
FILE.filter(length(FILE["COMMENTS"]) > 20).show()

+---------+--------------------+
|USER_NAME|            COMMENTS|
+---------+--------------------+
| Aliya153| Your website is ...|
|    Sara2| You need to work...|
|    Ali45| I will definitel...|
+---------+--------------------+



In [ ]:
counts = FILE.groupBy("USER_NAME").count()
counts.show()

+---------+-----+
|USER_NAME|count|
+---------+-----+
|    Ali45|    2|
| Aliya153|    1|
|    Sara2|    1|
+---------+-----+



In [ ]:
max_commenter = counts.orderBy(desc("count")).first()["USER_NAME"]
print("USER WITH MAXIMUM NUMBER OF COMMENTS=",max_commenter)

USER WITH MAXIMUM NUMBER OF COMMENTS= Ali45


#QUESTION_FOUR:

In [ ]:
file=spark.read.csv("english.txt")
file.show()

+-----------+
|        _c0|
+-----------+
|        'll|
|       'tis|
|      'twas|
|        've|
|         10|
|         39|
|          a|
|        a's|
|       able|
|  ableabout|
|      about|
|      above|
|     abroad|
|       abst|
| accordance|
|  according|
|accordingly|
|     across|
|        act|
|   actually|
+-----------+
only showing top 20 rows



In [ ]:
stop_words = StopWordsRemover.loadDefaultStopWords("english")

In [ ]:
def remove_stop_words(words):
    return [word for word in words if word.lower() not in stop_words]

In [ ]:
remove_stop_words_udf = udf(remove_stop_words, ArrayType(StringType()))

In [ ]:
df = FILE.withColumn("COMMENTS", remove_stop_words_udf("COMMENTS"))

In [ ]:
df.show()

+---------+--------------------+
|USER_NAME|            COMMENTS|
+---------+--------------------+
| Aliya153|[ , Y, o, u, r,  ...|
|    Sara2|[ , Y, o, u,  , n...|
|    Ali45|[ , G, o, o, d,  ...|
|    Ali45|[ ,  , w, l, l,  ...|
+---------+--------------------+

